### Task 1:
Дан массив чисел, состоящий из некоторого количества подряд идущих единиц, за которыми следует какое-то количество подряд идущих нулей: 111111111111111111111111100000000. 
Найти индекс первого нуля (то есть найти такое место, где заканчиваются единицы, и начинаются нули)

Какова сложность вашего алгоритма?


Можно выполнить задание с помощью встроенного метода для списков:

In [1]:
def task1_1(array):
    return array.index(0)

Но сложность этого метода -- O(n), т.к. метод работает на любых, необязательно отсортированных массивах.
В нашем случае массив отсортирован, поэтому можно применить бинарный поиск, сложность которого -- O(log(n)):

In [2]:
def task1_2(array):
    left = 0
    right = len(array) - 1
    while True:
        ind = (left + right) // 2
        if left == right and array[ind] == 1:
            break
        if array[ind] == 0:
            if ind == 0 or array[ind - 1] == 1:
                return ind
            else:
                right = ind - 1
        else:
            left = ind + 1
    return None

Проверим, что ответы совпадают на всех строках длины 100:

In [3]:
for i in range(100):
    s = [1] * i + [0] * (100 - i)
    assert task1_1(s) == task1_2(s)

### Task 2:
В нашей школе мы не можем разглашать персональные данные пользователей, но чтобы преподаватель и ученик смогли объяснить нашей поддержке, кого они имеют в виду (у преподавателей, например, часто учится несколько Саш), мы генерируем пользователям уникальные и легко произносимые имена. Имя у нас состоит из прилагательного, имени животного и двузначной цифры. В итоге получается, например, "Перламутровый лосось 77". Для генерации таких имен мы и решали следующую задачу:
Получить с русской википедии список всех животных (Категория:Животные по алфавиту) и вывести количество животных на каждую букву алфавита. Результат должен получиться в следующем виде:
 А: 642
Б: 412
В:....


С помощью BeautifulSoup парсим первую страницу, находим, в каком тэге находится список животных. В задаче требуется вывести только количество животных на каждую букву алфавита, но моя программа получает также список названий одновременно с подсчетом. Затем, пока названия животных состоят из кириллических букв, переходим на следующую страницу и обновляем список животных и словарь количества названий на каждую букву.

In [4]:
import requests
from bs4 import BeautifulSoup
import re

In [5]:
def parse_ul(ul, animals_list, letter_dict):
    for tag in ul.find_all("li", recursive=True):
        animal_name = tag.text
        if not re.search('[а-яА-Я]', animal_name):
            return (-1)
        first_letter = animal_name[0]
        animals_list.append(animal_name)
        if first_letter not in letter_dict:
            letter_dict[first_letter] = 0
        letter_dict[first_letter] += 1
    return (0)

In [6]:
def parse_wiki_animals():
    animals_list = []
    letter_dict = {}
    url = 'https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D1%82%D0%B5%D0%B3%D0%BE%D1%80%D0%B8%D1%8F:%D0%96%D0%B8%D0%B2%D0%BE%D1%82%D0%BD%D1%8B%D0%B5_%D0%BF%D0%BE_%D0%B0%D0%BB%D1%84%D0%B0%D0%B2%D0%B8%D1%82%D1%83'
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        s = soup.find_all("div", {"class":"mw-category-group"})
        answer = parse_ul(s[0].ul, animals_list, letter_dict)
        if answer == 0:
            url = 'https://ru.wikipedia.org/' + soup.find('a', text='Следующая страница')['href']
        else:
            break
    return (animals_list, letter_dict)

In [7]:
animals, letters = parse_wiki_animals()

Чтобы буква Ё находилась на своём месте, пришлось немного изменить функцию сортировки.

In [8]:
for key, value in sorted(letters.items(), key = lambda c: ord('Е') + 0.5 if c[0] == 'Ё' else ord(c[0])):
    print(f'{key}: {value}')

А: 1090
Б: 1395
В: 483
Г: 820
Д: 530
Е: 27
Ё: 2
Ж: 211
З: 395
И: 322
К: 2003
Л: 469
М: 1054
Н: 287
О: 618
П: 1462
Р: 390
С: 1653
Т: 766
У: 197
Ф: 169
Х: 222
Ц: 28
Ч: 456
Ш: 115
Щ: 56
Э: 51
Я: 171


Можно проверить список животных:

In [9]:
animals[1740:1760]

['Блестящая улитка',
 'Блестящегрудый усач',
 'Блестящехвостые колибри',
 'Блестящие златки',
 'Блестящие лори',
 'Блестящие скворцы',
 'Блестящие чирки',
 'Блестящий ворон',
 'Блестящий гульман',
 'Блестящий древесный уж',
 'Блестящий древолаз',
 'Блестящий дрозд',
 'Блестящий клехо',
 'Блестящий крючкоклюв',
 'Блестящий лиолемус',
 'Блестящий лори-кардинал',
 'Блестящий малый трупиал',
 'Блестящий муравей-древоточец',
 'Блестящий расписной малюр',
 'Блестящий фаэтончик']

### Task 3:
Мы сохраняем время присутствия каждого пользователя на уроке  виде интервалов. В функцию передается словарь, содержащий три списка с таймстемпами (время в секундах): 
— lesson – начало и конец урока 
— pupil – интервалы присутствия ученика 
— tutor – интервалы присутствия учителя 
Интервалы устроены следующим образом – это всегда список из четного количества элементов. Под четными индексами (начиная с 0) время входа на урок, под нечетными - время выхода с урока.
Нужно написать функцию, которая получает на вход словарь с интервалами и возвращает время общего присутствия ученика и учителя на уроке (в секундах). 


Чтобы найти промежутки, во время которых на уроке были и ученик, и учитель, можно написать функцию, которая для двух списков таймстемпов в заданном формате находит их пересечения, а затем пересечь ученика и учителя, а получившиеся таймстемпы - с началом и концом урока:

In [10]:
def intersect(pupil, tutor):
    p = 0
    t = 0
    intersections = []
    while True:
        if p > len(pupil) - 2 or t > len(tutor) - 2:
            break
        if pupil[p + 1] <= tutor[t]:
            p += 2
        elif tutor[t + 1] <= pupil[p]:
            t += 2
        else:
            intersections.append(max(pupil[p], tutor[t]))
            intersections.append(min(pupil[p + 1], tutor[t + 1]))
            if pupil[p + 1] == tutor[t + 1]:
                p += 2
                t += 2
            elif pupil[p + 1] < tutor[t + 1]:
                p += 2
            else:
                t += 2
    return intersections

На тестах стало понятно, что промежутки присутствия могут пересекаться внутри одного списка, поэтому их необходимо было объединить:

In [11]:
def clean_intervals(intrv_list):
    clean = []
    low = intrv_list[0]
    high = intrv_list[1]
    for i in range(2, len(intrv_list) - 1, 2):
        if intrv_list[i] <= high:
            high = max(high, intrv_list[i + 1])
        else:
            clean.append(low)
            clean.append(high)
            low = intrv_list[i]
            high = intrv_list[i + 1]
    clean.append(low)
    clean.append(high)
    return (clean)

Вызываем обе функции и считаем сумму нужного нам времени:

In [12]:
def appearance(intervals):
    p = clean_intervals(intervals['pupil'])
    t = clean_intervals(intervals['tutor'])
    p_t_intersect = intersect(p, t)
    total_intersect = intersect(p_t_intersect, intervals['lesson'])
    total = 0
    for i in range(0, len(total_intersect) - 1, 2):
        total += (total_intersect[i + 1] - total_intersect[i])
    return total

In [13]:
tests = [
   {'data': {'lesson': [1594663200, 1594666800],
             'pupil': [1594663340, 1594663389, 1594663390, 1594663395, 1594663396, 1594666472],
             'tutor': [1594663290, 1594663430, 1594663443, 1594666473]},
    'answer': 3117
    },
   {'data': {'lesson': [1594702800, 1594706400],
             'pupil': [1594702789, 1594704500, 1594702807, 1594704542, 1594704512, 1594704513, 1594704564, 1594705150, 1594704581, 1594704582, 1594704734, 1594705009, 1594705095, 1594705096, 1594705106, 1594706480, 1594705158, 1594705773, 1594705849, 1594706480, 1594706500, 1594706875, 1594706502, 1594706503, 1594706524, 1594706524, 1594706579, 1594706641],
             'tutor': [1594700035, 1594700364, 1594702749, 1594705148, 1594705149, 1594706463]},
    'answer': 3577
    },
   {'data': {'lesson': [1594692000, 1594695600],
             'pupil': [1594692033, 1594696347],
             'tutor': [1594692017, 1594692066, 1594692068, 1594696341]},
    'answer': 3565
    },
]

if __name__ == '__main__':
    for i, test in enumerate(tests):
        test_answer = appearance(test['data'])
        assert test_answer == test['answer'], f'Error on test case {i}, got {test_answer}, expected {test["answer"]}'


### Также эту функцию можно протестировать через API, запустить который можно из файла task3.py